### **Exploración y preprocesamiento de datos**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el archivo CSV
dataframe = pd.read_csv('anemia.csv')
print(dataframe)

     Number Sex  %Red Pixel  %Green pixel  %Blue pixel    Hb Anaemic
0         1   M     43.2555       30.8421      25.9025   6.3     Yes
1         2   F     45.6033       28.1900      26.2067  13.5      No
2         3  F      45.0107       28.9677      26.0215  11.7      No
3         4   F     44.5398       28.9899      26.4703  13.5      No
4         5  M      43.2870       30.6972      26.0158  12.4      No
..      ...  ..         ...           ...          ...   ...     ...
99      100  F      49.9999       29.2860      20.7141  14.5     Yes
100     101  M      42.2324       30.6757      27.0919   6.3     Yes
101     102   F     45.6064       31.9084      22.4852  12.7      No
102     103  M      45.2095       29.2769      25.5136  13.4      No
103     104   F     43.5706       29.8094      26.6199  12.2      No

[104 rows x 7 columns]


### **Modelo y evaluación**